<table>
 <tr align=left><td><img align=left src="https://i.creativecommons.org/l/by/4.0/88x31.png">
 <td>Text provided under a Creative Commons Attribution license, CC-BY. All code is made available under the FSF-approved MIT license. (c) Kyle T. Mandli</td>
</table>

In [ ]:
from __future__ import print_function

%matplotlib inline
import numpy
import matplotlib.pyplot as plt

# Stochastic Spectral Methods

**Purpose:** Construct surrogate (or Reduce Order Models) to help reduce the cost of UQ approaches.

Examples:
 - Bayesian model calibration, sensitivity analysis, design and control.
 - Posterior density sampling

**Key Idea:** Exploit smoothness of high-dimensional parameter spaces.

## Spectral Representation of Random Processes

Some definitions:

- Define a sequence of random variables
$$
    \left \{ Q_k(\omega) \right \}^\infty_{k=1}
$$
on the sample space $\Omega$ and probability space $(\Omega, \mathcal F, P)$.

- Let $\mathbb P_k$ denote space or polynomials with argument $Q_i$ with degree $\leq k$.
- Let $\hat{\mathcal P}_k \in \mathbb P_k$ s.t. $P \in \hat{\mathcal P}_k$ are orthogonal to $\mathbb P_{k-1}$.

### Polynomial Expansions

For a second order, finite-variance random variable $u$ can be represented as
$$
    u(\omega) = u_0 \hat{\!P}_0 + \sum^\infty_{i_1 = 1} u_{i_1} \hat{\!P}_1(Q_{i_1}) + \sum^\infty_{i_1 = 1} \sum^\infty_{i_2 = 1} u_{i_1, i_2} \hat{\!P}_2(Q_{i_1}, Q_{i_2}) + \sum^\infty_{i_1 = 1} \sum^\infty_{i_2 = 1} \sum^\infty_{i_4 = 1} u_{i_1, i_2, i_3} \hat{\!P}_3(Q_{i_1}, Q_{i_2}, Q_{i_3}) + \cdots
$$
Here $ \hat{\!P}_k(\cdot)$ represent the interaction between variables and $u_{i_1}, u_{i_1, i_2}, \ldots \in \mathbb R$.

More succinctly this can be written as
$$
    u(Q) = \sum^\infty_{k=0} u_k \Psi_k(Q_1, Q_2, Q_3, \ldots)
$$

If we have a finite set of $\left \{Q_i\right\}^p_{k=1}$ then we have a finite sum that can represent $u(\omega)$ up to $K$-order interactions as
$$
    u^K(Q) = \sum^K_{k=0} u_k \Psi_k(Q_1, Q_2, Q_3, \ldots)
$$
where $K+1 = \frac{(n+p)!}{n!p!}$.

For example if the random variable $u$ has only second-order interactions and less we would have
$$
u(\omega) = u_0 \hat{\!P}_0 + \sum^\infty_{i_1 = 1} u_{i_1} \hat{\!P}_1(Q_{i_1}) + \sum^\infty_{i_1 = 1} \sum^\infty_{i_2 = 1} u_{i_1, i_2} \hat{\!P}_2(Q_{i_1}, Q_{i_2})
$$

Now consider a random process $u(t, x, \omega)$ where now we have also added possible time and spatial dependence.  Our polynomial representation then can be written as
$$
    u^K(t, x, Q) = \sum^K_{k=0} u_k(t, x) \Psi_k(Q)
$$
where we note that we have separated the time and spatial dependence from the random variables.  The $\Psi_k$ are then usually looked as a orthogonal polynomial basis for the random part of the process $u$.

**Example:**

Consider a single random variable $Q \in C_0$ and take $\Psi_k(Q)$ as some set of one-dimensional polynomials that are orthogonal to each other with respect to the density $\rho_Q(q)$ and normalized so that $\Psi_0 = 1$.  Then
$$
    \mathbb E[\psi_0(Q)] = 1
$$
and
$$\begin{aligned}
    \mathbb E[\psi_i(Q) \psi_j(Q)] &= \int \psi_i(Q) \psi_j(Q) \rho_Q(q) dq \\
    &= \langle \psi_i(Q) \psi_j(Q) \rangle_\rho \\
    &= \delta_{ij} \gamma_i
\end{aligned}$$

Note that this is analogous to projecting onto an orthogonal polynomial basis.  Compute the mean and variance of the process $u(t, x, \omega)$ given the above rules.

**Mean:**
$$\begin{aligned}
    \mathbb E [u^K(t, x, Q)] &= \mathbb E \left [ \sum^K_{k=0} u_k(t, x) \psi_k(Q) \right ] \\
    &= u_0(t, x) \mathbb E[\psi_0(Q)] + \sum^K_{k=1} u_k(t, x) \mathbb E[\psi_k(Q)] \\
    &= u_0(t, x)
\end{aligned}$$

**Variance:**
$$\begin{aligned}
    \text{var} [u^K(t, x, Q)] &= \mathbb E \left [\left ( u^K(t, x, Q) - \mathbb E[u^K(t, x, Q)] \right )^2 \right] \\
    &= \mathbb E \left [\left(\sum^K_{k=0} u_k(t, x) \psi_k(Q) - u_0(t,x) \right )^2 \right ] \\
    &= \mathbb E \left [\left(u_0(t,x) + \sum^K_{k=1} u_k(t, x) \psi_k(Q) - u_0(t,x) \right )^2 \right ] \\
    &= \mathbb E \left [\left(\sum^K_{k=1} u_k(t, x) \psi_k(Q) \right )^2 \right ] \\
    &= \sum^K_{k=1} u^2_k(t, x)\gamma_k
\end{aligned}$$

### Basis for Distributions

As mentioned we want to have a polynomial basis that is orthogonal w.r.t. a density.

**Normal Distribution** $Q \sim N(0, 1)$
$$
    \rho_Q(q) = \frac{1}{\sqrt{2 \pi}} e^{-q^2 / 2}
$$
defined on $\mathbb R$, use the Hermite polynomials:
$$\begin{aligned}
    &H_0(Q) = 1, & & H_1(Q) = Q, & & H_2(Q) = Q^2 - 1, \\
    &H_3(Q) = Q^3 - 3 Q, & & H_4(Q) = Q^4 - 6Q^2 + 3, & & H_5(Q) = Q^5 - 10 Q^3 + 15 Q, \\
\end{aligned}$$

Normalization constants:
$$
    \gamma_i = \int_{\mathbb R} \psi^2(q) \rho_Q(q)dq = i!
$$

**Uniform Distribution** $Q \sim \mathcal{U}(-1, 1)$
$$
    \rho_Q(q) = \frac{1}{2}
$$
defined on $[-1, 1]$.
For a uniform distribution we use the Legendre polynomials:
$$\begin{aligned}
    &P_0(Q) = 1, & & P_1(Q) = Q, & & P_2(Q) = \frac{3}{2} Q^2 - \frac{1}{2}, \\
    &P_3(Q) = \frac{5}{2} Q^3 - \frac{3}{2} Q, & & P_4(Q) = \frac{35}{8} Q^4 - \frac{15}{4} Q^2 + \frac{3}{8}, & & P_5(Q) = \frac{63}{8} Q^5 - \frac{70}{8} Q^3 + \frac{15}{8} Q, \\
\end{aligned}$$

**Example:** Take $u \sim N(\mu, \sigma^2)$ that can be represented as
$$
    u = \mu + \sigma Q
$$
where $Q \sim N(0, 1)$.  Compute the coefficients of the polynomial representation of $u$.

$$
    \mu = \mathbb E \left [ \sum^K_{k=0} u_k(t, x) \psi_k(Q) \right ] = u_0(t, x) \Rightarrow u_0 = \mu
$$

$$\begin{aligned}
    \sigma^2 &= \mathbb E \left [\left ( u^K(t, x, Q) - \mathbb E[u^K(t, x, Q)] \right )^2 \right] \\
    &= \sum^K_{k=1} u^2_k(t, x) \gamma_k \\
    &= u^2_1(t, x) \gamma_1 \Rightarrow u_1 = \sigma
\end{aligned}$$

**Example:** Take $u \sim \mathcal U(a, b)$ that has mean and variance
$$
    \mu = \frac{a+b}{2} \quad \quad \sigma^2 = \frac{(b - a)^2}{12}
$$
and can be expressed as
$$
    u = \mu + \sqrt{3} \sigma Q
$$
where $Q \sim \mathcal{U}(-1, 1)$.  Compute the coefficients of the polynomial representation of $u$.

Similarly to the case of a normal distribution we have
$$
    u_0 = \mu = \frac{a + b}{2}
$$
and
$$\begin{aligned}
    u_1 = \sqrt{3} \sigma
\end{aligned}$$

### Multiple Random Variables

The single random variable case naturally extends to multiple random variables if we assume that the variables are assumed to be independent of each other (not necessarily the case).  This implies that the expectation of their product is the expectation of the individual variables multiplied together and motivates the following.

A *p-dimensional Multi-Index* is a $p$-tuple where
$$
    \boldsymbol{k'} = (k_1, \ldots, k_p) \in \mathbb N^p_0
$$
of non-negative integers with magnitude
$$
    |\boldsymbol{k'}| = \sum^p_{i=1} k_i
$$
and are ordered such that
$$
    \boldsymbol{j'} \leq \boldsymbol{k'} \iff j_i \leq k_i \text{  for  } i=1, \ldots, p.
$$
This is a bit hard to deal with but table 10.1 provides some values for the first few multi-indices as

| $k$ | $|\boldsymbol{k'}|$ | Multi-Index | Polynomial Multiplication              |
|----------------------------------------------------------------------------------|
|0    | 0                   | (0, 0, 0)   | $\psi_0(Q_1) \psi_0(Q_2) \psi_0(Q_3)$  |
|1    | 1                   | (1, 0, 0)   | $\psi_1(Q_1) \psi_0(Q_2) \psi_0(Q_3)$  |
|2    |                     | (0, 1, 0)   | $\psi_0(Q_1) \psi_1(Q_2) \psi_0(Q_3)$  |
|3    |                     | (0, 0, 1)   | $\psi_0(Q_1) \psi_0(Q_2) \psi_1(Q_3)$  |
|4    | 2                   | (2, 0, 0)   | $\psi_2(Q_1) \psi_0(Q_2) \psi_1(Q_3)$  |
|5    |                     | (1, 1, 0)   | $\psi_1(Q_1) \psi_1(Q_2) \psi_1(Q_3)$  |
|6    |                     | (1, 0, 1)   | $\psi_1(Q_1) \psi_0(Q_2) \psi_1(Q_3)$  |
|7    |                     | (0, 2, 0)   | $\psi_0(Q_1) \psi_2(Q_2) \psi_0(Q_3)$  |
|8    |                     | (0, 1, 1)   | $\psi_0(Q_1) \psi_1(Q_2) \psi_1(Q_3)$  |
|9    |                     | (0, 0, 2)   | $\psi_0(Q_1) \psi_0(Q_2) \psi_2(Q_3)$  |

Now define a vector of random variables 
$$
    Q = [Q_1,\ldots,Q_p]
$$ 
that are mutually independent with the density
$$
    \rho_Q = \prod^p_{i=1} \rho_{Q_p}.
$$

Let the univariate basis functions of each $Q_i$ be
$$
    \left \{ \psi_k(Q_i) \right \}^K_{k=0}
$$
is the univariate basis functions of degree $\leq K$ for variable $Q_i$.  We can then form the multivariate basis as
$$
    \Psi_{\boldsymbol{i'}}(Q) = \psi_{i_1}(Q_1), \cdots \psi_{i_p}(Q_p)
$$
for $0 \leq |\boldsymbol{i'}| \leq K$.  

The resulting basis functions therefore satisfy
$$\begin{aligned}
    \mathbb E[\Psi_{\boldsymbol{i'}}(Q) \Psi_{\boldsymbol{j'}}(Q)] &= \int \Psi_{\boldsymbol{i'}}(q) \Psi_{\boldsymbol{j'}}(q) \rho_Q(q) dq \\
    &= \langle \Psi_{\boldsymbol{i'}}, \Psi_{\boldsymbol{j'}} \rangle_\rho \\
    &= \delta_{\boldsymbol{i'} \boldsymbol{j'}} \gamma_{\boldsymbol{i'}}
\end{aligned}$$
where
$$
    \gamma_{\boldsymbol{i'}} = \mathbb{E}[\Psi^2_{\boldsymbol{i'}}] = \gamma_{i_1} \cdots \gamma_{i_p}.
$$

Turning back now to the representation of our process we have for $u(t, x, Q)$ the expansion
$$
    u^K(t, x, Q) = \sum^K_{|\boldsymbol{k'}| = 0} u_{\boldsymbol{k'}}(t, x) \Psi_{\boldsymbol{k'}}(Q),
$$
again the projection of u(t, x, Q) onto the basis $\Psi_{\boldsymbol{k'}}$.  Moreover the orthogonality of the basis functions allow us to write down
$$
    u_k(t,x) = \frac{1}{\gamma_k} \mathbb E[u(t, x, Q) \Psi_k(Q) ]
$$

Moreover the orthogonality of the basis functions allow us to write down
$$
    u_k(t,x) = \frac{1}{\gamma_k} \mathbb E[u(t, x, Q) \Psi_k(Q) ]
$$

## Galerkin, Collocation, and Discrete Projection Frameworks

We now turn to ways to compute the $u_k(t,x) using constraints provided by the assumptions of each approach.

### Finite Elements

As an aside we will briefly describe the relatively similar notation and ideas from finite elements and how they will relate to the methods for computing the coefficients $u_k(t, x)$.

Consider the simple ODE
$$
    \frac{\text{d}^2 u}{\text{d}x^2} = f(x) \quad x \in \Omega \quad u|_{\partial \Omega} = \Gamma(x).
$$

The first thing we will do is write the equation above, a.k.a. the *strong-form* of the equation, in the *weak-form* instead.  We do this by multiplying by a test function $v$ that satisfies the boundary conditions $v|_{\partial \Omega} = 0$ (you can also require $u$ to do this) and integrating to find
$$
    \int_{\Omega} u''(x) v(x) dx = \int_{\Omega} v(x) f(x) dx.
$$

Integrating the LHS by parts leads to
$$\begin{aligned}
    u'(x) v(x) |_{\partial \Omega} - \int_{\Omega} u'(x) v'(x) dx &= \int_{\Omega} v(x) f(x) dx. \\
    - \int_{\Omega} u'(x) v'(x) dx &= \int_{\Omega} v(x) f(x) dx.
\end{aligned}$$

Since the weak-form of the equation should be true $\forall v \in H^1_0(\Omega)$ such that we can restate the problem as 
$$
    \text{find a } u \in H^1_0(\Omega) \quad \forall v \in H^1_0(\Omega) \quad \int_{\Omega} u'(x) v'(x) dx = \int_{\Omega} v(x) f(x) dx.
$$
This is an infinite dimensional problem and where discretization occurs.  Instead of the above problem we replace the Sobolev space with a finite dimensional space $V$ such that the problem is now to
$$
    \text{find a } u \in U \quad \forall v \in V \quad \int_{\Omega} u'(x) v'(x) dx = \int_{\Omega} v(x) f(x) dx.
$$

For finite element methods we generally pick compactly support (with regards to the domain) piece-wise defined polynomials such as the hat functions
$$
    \psi_i(x) = \left \{ \begin{aligned}
        &\frac{x - x_{i-1}}{x_i - x_{i-1}} & & \text{if } x \in [x_{i-1}, x_i] \\
        &\frac{x_{i+1} - x}{x_{i+1} - x_{i}} & & \text{if } x \in [x_{i}, x_{i+1}] \\
        &0 & & \text{otherwise}
    \end{aligned} \right .
$$
Note with this example that the basis is orthogonal at the nodes of the grid $x_i$ and have overlapping support in the intervals between nodes.  Other choices, such as the Fourier basis, lead to other methods such as spectral methods.

The final element (heh) of turning the problem above into a discretization is to write the solution $u$ as
$$
    u \approx U = \sum^K_{k=0} u_k \psi_k(x)
$$
or in other words assume that the $\psi_k(x) \in \Psi$ spans the space $U$.

Plugging this back into the weak form we have
$$\begin{aligned}
    \int_{\Omega} u'(x) v'(x) dx &= \int_{\Omega} v(x) f(x) dx \\
    \int_{\Omega}  v'(x) \sum^K_{k=0} u_k  \psi_k'(x) dx &= \int_{\Omega} v(x) f(x) dx
\end{aligned}$$

We now have to identify the space $V$.  A Galerkin method (also sometimes called model finite elements) assumes that the same set of basis functions also spans the space $V$ (or in most cases that $U = V$).  This then turns the weak form into
$$\begin{aligned}
    \int_{\Omega}  \psi_j'(x) \sum^K_{k=0} u_k  \psi_k'(x) dx &= \int_{\Omega} \psi_j(x) f(x) dx \quad \forall \psi_j \in \Psi \\
    \sum^K_{k=0} u_k \int_{\Omega}  \psi_j'(x) \psi_k'(x) dx &= \int_{\Omega} \psi_j(x) f(x) dx \quad \forall \psi_j \in \Psi
\end{aligned}$$
for suitable assumptions.  This last expression can then be understood as a matrix problem on the LHS with the entries in the matrix comprised of
$$
    A_{jk} = \int_{\Omega} \psi_j'(x) \psi_k'(x) dx
$$
and the RHS the projection of $f(x)$ onto the space $\Psi$ giving us the discretized problem
$$
    A U = f.
$$

One way to think of the finite dimensional problem is to think of the problem as we have defined it as a projection onto the function space $U$ and $V$.  The function $f(x)$ is being projected onto $V$ and $U$ defines the space of functions we can look at to solve the problem (the search space).  If a problem converges to the true solution then $U \rightarrow H$ where $H$ contains the true solution.

There are three different ways we will approach this problem:
 - *Stochastic Galerkin Approach:*  This proceeds as we did with the finite element discussion and is equivalent to minimizing the residual onto a finite subspace.  Unfortunately things are not so easy when we switch back to our spectral representation of our process.  Instead the projection now requires the computation of expectations that are not usually needed in a deterministic setting.  This is often then called an *intrusive* method.
 - *Collocation:*  An alternative to Galerkin approaches is collocation or nodal methods.  Here we approximate the solution at a discrete set of points, called nodes or collocation points, and a space of polynomials $\Psi$ such that the problem we try to solve is what $\psi \in \Psi$ approximates the solution at the nodes the best.  This approach is considered *non-intrusive* as existing collocation approaches can be used.  Note that collocation approaches can be thought of as a specific type of Galerkin method.
 - *Discrete Projection:*  Direct approximation of the integral for the coefficients is used and can often be seen as another form of a Galerkin approach.  This approach often needs to be implemented independently but not always.
 
We now turn to studying these three methods applied to problems of varying complexity via examples.

## Examples

### Scalar Initial Value Problem

Consider the initial value problem
$$
    \frac{\text{d} u}{\text{d}t} = f(t, Q, u) \quad t > 0 \quad u(0, Q) = u_0.
$$
Assume $Q = [Q_1,\ldots,Q_p]$ are mutually independent random variables with range $\Gamma \in \mathbb R^p$ and joint density $\rho_Q(q)$.  Take the QoI as
$$
    y(t) = \int_{\Gamma} u(t, q) \rho_Q(q) dq
$$
and solutions $u \in L^2(0, T)$.  We also assume that we will be dealing with spaces of functions that have finite norm w.r.t. the $i$th component of the density such that if
$$
    ||g||_2 = \left(\int_{\Gamma_i} |g(q_i)|^2 \rho_{Q_i}(q_i) dq_i \right)^{1/2} < \infty.
$$
then $g \in L^2_{\rho_i}(\Gamma_i)$.  We can then consider the composed space of functions 
$$
    L^2_\rho(\Gamma) = L^2_{\rho_1}(\Gamma_1) \otimes \cdots \otimes L^2_{\rho_p}(\Gamma_p).
$$

Let $\{\Psi_k\}^K_{k=1}$ be a basis of a finite subspace $Z^K \subset L^2_\rho(\Gamma)$ such that we can project $u(t, Q)$ onto $Z^K$ to find
$$
    u^K(t, Q) = \sum^K_{k=0} u_k(t) \Psi_k(Q).
$$
The resulting coefficients can be computed by computing
$$
    u_k(t) = \frac{1}{\gamma_k} \int_\Gamma u(t, q) \Psi_k(q) \rho_Q(q) dq.
$$
We can now turn to finding ways to approximate the above integral for the coefficients.

#### Stochastic Galerkin

#### Collocation

#### Discrete Projection

### Elliptic PDEs

#### Stochastic Galerkin

#### Collocation

#### Discrete Projection

### Evolution PDEs

#### Stochastic Galerkin

#### Collocation

#### Discrete Projection

### Additional Reading
 - *Spectral Methods for Uncertainty Quantification* by O.P. Le Maitre and O.M. Knio